# Building Your First Campaign

This notebook will walk you through the steps to build a recommendation model for recommending items to users based on the device they use. The goal is to recommend items that are relevant based on a particular user.

The data is created using Python's Faker Library (https://faker.readthedocs.io/en/master/).

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) which are core data science tools.

In [ ]:
!pip install Faker

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import csv
import sys
import random

from faker import Faker
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from random import randint, randrange

Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
cf_client = boto3.client('cloudformation')
ssm_client = boto3.client('ssm')

In [ ]:
STACK_NAME = 'personalize-auto-context-stack' # if you're not using the default stack name, replace this
%store STACK_NAME

In [ ]:
try:
    describe_stack_resource_response = cf_client.describe_stack_resource(
        StackName=STACK_NAME,
        LogicalResourceId='PersonalizeAutoContextBucket'
    )
    PersonalizeAutoContextBucket = describe_stack_resource_response["StackResourceDetail"]["PhysicalResourceId"]
    print('S3 Bucket is:', PersonalizeAutoContextBucket)
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [ ]:
bucket =PersonalizeAutoContextBucket     # replace with the name of your S3 bucket
interactions_filename = "personalize-auto-context-input-interactions.csv"
items_filename = "personalize-auto-context-input-items.csv"

In [ ]:
three_months = date.today() + relativedelta(months=-3)
print(three_months)
print(date.today())

In [ ]:
def datagenerate(records, headers):
    fake = Faker('en_US')
    item_interaction_mode = {}
    with open(interactions_filename, 'wt') as csvFile:
        writer = csv.DictWriter(csvFile, fieldnames=headers)
        writer.writeheader()
        for i in range(records):
            user_id = fake.pyint(min_value=100, max_value=999, step=30)
            item_id = fake.pyint(min_value=100, max_value=999, step=20)
            if (item_id in item_interaction_mode):
                device_type = item_interaction_mode[item_id]
            else:
                device_type = fake.word(ext_word_list=['Desktop', 'Phone', 'TV', 'Tablet'])
                item_interaction_mode[item_id]=device_type
            # interaction_timestamp = fake.unix_time(end_datetime=datetime(2023,3,31), start_datetime=datetime(2023,1,1))
            interaction_timestamp = fake.unix_time(end_datetime=date.today(), start_datetime=three_months)
            writer.writerow({
                    "USER_ID" : user_id,
                    "ITEM_ID" : item_id,
                    "DEVICE_TYPE": device_type,
                    "TIMESTAMP" : interaction_timestamp
                    })

In [ ]:
records = 10000
headers = ["USER_ID", "ITEM_ID", "DEVICE_TYPE", "TIMESTAMP"]
datagenerate(records, headers)

In [ ]:
def datagenerate_items(headers):
    fake = Faker('en_US')
    with open(interactions_filename, mode="r") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)
        with open(items_filename, "w") as output:
            writer = csv.DictWriter(output, fieldnames=headers)
            writer.writeheader()
            item_interaction_mode = {}
            for item in reader:
                if item[2] == 'Desktop':
                    genre = fake.word(ext_word_list=['Documentary', 'Historical', 'Science Fiction'])
                elif item[2] == 'Phone':
                    genre = fake.word(ext_word_list=['Sitcom', 'Comedy', 'Satire'])
                elif item[2] == 'TV':
                    genre = fake.word(ext_word_list=['Action', 'Horror', 'Romance', 'Mystery'])
                elif item[2] == 'Tablet':
                    genre = fake.word(ext_word_list=['Sitcom', 'Comedy', 'Satire', 'Documentary'])
                else:
                    genre = 'Fiction'
            
                item_id_int = int(item[1])
        
                if item_id_int % 3 == 0:
                    allowed_countries = 'US|PK|NI|JM|IN|DK'
                elif item_id_int % 3 == 1:
                    allowed_countries = 'US|FI|CN|ES|HK|AE'
                elif item_id_int % 3 == 2:
                    allowed_countries = 'RU|GR|LT|NO|SZ|VN'
                else:
                    allowed_countries = 'VA|UA|TH|NE|MC|IE'
                creation_timestamp = fake.unix_time(end_datetime=date.today(), start_datetime=three_months)
                
                if (item[1] not in item_interaction_mode):
                    writer.writerow({
                        "ITEM_ID" : item[1],
                        "GENRE" : genre,
                        "ALLOWED_COUNTRIES" : allowed_countries,
                        "CREATION_TIMESTAMP" : creation_timestamp
                    })
                    item_interaction_mode[item[1]]=genre

In [ ]:
headers = ["ITEM_ID", "GENRE", "ALLOWED_COUNTRIES", "CREATION_TIMESTAMP"]
datagenerate_items(headers)

## Configure the data

Data is imported into Amazon Personalize through Amazon S3, below we will specify a bucket that you have created within AWS for the purposes of this exercise.

Below you will update the `bucket` variable to instead be set to the value that you created earlier in the CloudFormation steps, this should be in a text file from your earlier work. the `filename` does not need to be changed.

#### Explore the Dataset

In [ ]:
interactions_data = pd.read_csv(interactions_filename, header=0)
pd.set_option('display.max_rows', 5)
interactions_data

In [ ]:
items_data = pd.read_csv(items_filename, header=0)
pd.set_option('display.max_rows', 5)
items_data

#### Prepare and Upload Data

As you can see the data contains a UserID, ItemID, Rating, and Timestamp.

We are now going to remove the items with low rankings, and remove the Rating column before we build our model.

Once done we will now save the file as a new CSV and then upload it to S3.

All of that is done by executing the lines in the cell below.

In [ ]:
interactions_data.to_csv(interactions_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).upload_file(interactions_filename)

In [ ]:
items_data.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(items_filename).upload_file(items_filename)

### Create Schema

A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "DEVICE_TYPE",
            "type": "string",
            "categorical": True
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_interactions_schema_response = personalize.create_schema(
    name = "personalize-auto-context-demo-interactions-schema",
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(json.dumps(create_interactions_schema_response, indent=2))

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },
        {
            "name": "ALLOWED_COUNTRIES",
            "type": "string",
            "categorical": True
        },
        {
            "name": "CREATION_TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_items_schema_response = personalize.create_schema(
    name = "personalize-auto-context-demo-items-schema",
    schema = json.dumps(items_schema)
)

items_schema_arn = create_items_schema_response['schemaArn']
print(json.dumps(create_items_schema_response, indent=2))

### Create and Wait for Dataset Group

The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like.

#### Create Dataset Group

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-auto-context-demo-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Wait for Dataset Group to Have ACTIVE Status

Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

#### Create Dataset

After the group, the next thing to create is the actual datasets, in this example we will only create 1 for the interactions data. Execute the cells below to create it.

In [ ]:
dataset_type = "INTERACTIONS"
create_interactions_dataset_response = personalize.create_dataset(
    name = "personalize-auto-context-demo-interactions-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_interactions_dataset_response['datasetArn']
print(json.dumps(create_interactions_dataset_response, indent=2))

In [ ]:
dataset_type = "ITEMS"
create_items_dataset_response = personalize.create_dataset(
    name = "personalize-auto-context-demo-items-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_items_dataset_response['datasetArn']
print(json.dumps(create_items_dataset_response, indent=2))

#### Attach Policy to S3 Bucket

Amazon Personalize needs to be able to read the content of your S3 bucket that you created earlier. The lines below will do that.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### Create Personalize Role

Also Amazon Personalize needs the ability to assume Roles in AWS in order to have the permissions to execute certain tasks, the lines below grant that.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(30) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Import the data

Earlier you created the DatasetGroup and Dataset to house your information, now you will execute an import job that will load the data from S3 into Amazon Personalize for usage building your model.

#### Create Dataset Import Job

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-auto-context-demo-interactions-dataset-import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

#### Wait for Dataset Import Job to Have ACTIVE Status

It can take a while before the import job completes, please wait until you see that it is active below.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_interactions_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    status = describe_interactions_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-auto-context-demo-items-dataset-import",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

items_dataset_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_items_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = items_dataset_import_job_arn
    )
    status = describe_items_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [ ]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

#### User Personalization
The [User-Personalization](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) (aws-user-personalization) recipe is optimized for all USER_PERSONALIZATION recommendation scenarios. When recommending items, it uses automatic item exploration.

With automatic exploration, Amazon Personalize automatically tests different item recommendations, learns from how users interact with these recommended items, and boosts recommendations for items that drive better engagement and conversion. This improves item discovery and engagement when you have a fast-changing catalog, or when new items, such as news articles or promotions, are more relevant to users when fresh.

You can balance how much to explore (where items with less interactions data or relevance are recommended more frequently) against how much to exploit (where recommendations are based on what we know or relevance). Amazon Personalize automatically adjusts future recommendations based on implicit user feedback.

First, select the recipe by finding the ARN in the list of recipes above.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-auto-context-demo-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Create Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Wait for Solution Version to Have ACTIVE Status

This will take approximately 15-20 minutes.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(45)

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more complex datasets you should see improvements.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

- *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
- *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
- *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:

- coverage: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
- mean_reciprocal_rank_at_25: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
- normalized_discounted_cumulative_gain_at_K: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
- precision_at_K: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum transactions per second (TPS) value (`minProvisionedTPS`). This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this demo, the minimum throughput threshold is set to 1. For more information, see the [pricing](https://aws.amazon.com/personalize/pricing/) page.

As mentioned above, the user-personalization recipe used for our solution supports automatic exploration of "cold" items. You can control how much exploration is performed when creating your campaign. The `itemExplorationConfig` data type supports `explorationWeight` and `explorationItemAgeCutOff` parameters. Exploration weight determines how frequently recommendations include items with less interactions data or relevance. The closer the value is to 1.0, the more exploration. At zero, no exploration occurs and recommendations are based on current data (relevance). Exploration item age cut-off determines items to be explored based on time frame since latest interaction. Provide the maximum item age, in days since the latest interaction, to define the scope of item exploration. The larger the value, the more items are considered during exploration. For our campaign below, we'll specify an exploration weight of 0.5.

#### Create Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-auto-context-demo-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.5"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Wait for Campaign to Have ACTIVE Status

This should take about 10 minutes.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

## Create a Filter

In [ ]:
create_filter_response = personalize.create_filter(
    name = 'personalize-auto-context-demo-country-filter',
    datasetGroupArn = dataset_group_arn,
    filterExpression = 'INCLUDE ItemID WHERE Items.ALLOWED_COUNTRIES IN ($CONTEXT_COUNTRY)'
) 
filter_arn = create_filter_response["filterArn"]
print("Filter ARN: " + filter_arn)

In [ ]:
%%time
max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    version_response = personalize.describe_filter(
        filterArn = filter_arn
    )
    status = version_response["filter"]["status"]
    if status == "ACTIVE":
        print("Build succeeded for {}".format(filter_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(filter_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The Filter build is still in progress')       
    time.sleep(15)

## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. 

In [ ]:
# Getting a random user:
user_id, item_id, device_type, _ = interactions_data.sample().values[0]
print("USER: {}".format(user_id))
%store user_id

In [ ]:
device_list = ['Phone', 'Desktop', 'TV', 'Tablet']
device = random.choice(device_list)
print("DEVICE: {}".format(device))
%store device

Using the user that you obtained above, the lines below will get recommendations for you and return the list of items that are recommended.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    context={
        'DEVICE_TYPE': device
    }
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

print('ITEM_ID'.ljust(8, ' '), 
      'GENRE'.ljust(20, ' '), 
      'ALLOWED_COUNTRIES'.ljust(20, ' '))

for item in item_list:
    key = item['itemId']
    df2=items_data.query(f"ITEM_ID == {key}")
    print(str(df2['ITEM_ID'].values[0]).ljust(8, ' '), 
          df2['GENRE'].values[0].ljust(20, ' '), 
          df2['ALLOWED_COUNTRIES'].values[0].ljust(20, ' '))

Let us identify a cold user (user not in input file) and try to get recommendations for them. Try to rerun below 3 cells repeatedly in order to see how recommendations are served for different cold users.

In [ ]:
cold_user_found = False
cold_user_id = 0

while not(cold_user_found):
    cold_user_temp = randint(100, 999)
    with open(interactions_filename, 'rt') as f:
        val_status = False
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            if cold_user_temp == row[0]: # if the username shall be on column 3 (-> index 2)
                val_status = True
    if not(val_status):
        cold_user_found = True
        cold_user_id = cold_user_temp
    else:
        cold_user_found = False


print("COLD USER: {}".format(cold_user_id))
%store cold_user_id

In [ ]:
cold_user_device_list = ['Phone', 'Desktop', 'TV', 'Tablet']
cold_user_device = random.choice(cold_user_device_list)
print("COLD USER DEVICE: {}".format(cold_user_device))
%store cold_user_device

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(cold_user_id),
    context={
        'DEVICE_TYPE': cold_user_device
    }
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", cold_user_id)

item_list = get_recommendations_response['itemList']

print('ITEM_ID'.ljust(8, ' '), 
      'GENRE'.ljust(20, ' '), 
      'ALLOWED_COUNTRIES'.ljust(20, ' '))

for item in item_list:
    key = item['itemId']
    df2=items_data.query(f"ITEM_ID == {key}")
    print(str(df2['ITEM_ID'].values[0]).ljust(8, ' '), 
          df2['GENRE'].values[0].ljust(20, ' '), 
          df2['ALLOWED_COUNTRIES'].values[0].ljust(20, ' '))

## Get Sample Recommendations using the Filter

After the campaign is active you are ready to get recommendations using the country filter.

In [ ]:
context_country = 'US'
print(context_country)

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    context={
        'DEVICE_TYPE': device
    },
    filterValues={"CONTEXT_COUNTRY" : f"\"{context_country}\""}
)

# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

print('ITEM_ID'.ljust(8, ' '), 
      'GENRE'.ljust(20, ' '), 
      'ALLOWED_COUNTRIES'.ljust(20, ' '))

for item in item_list:
    key = item['itemId']
    df2=items_data.query(f"ITEM_ID == {key}")
    print(str(df2['ITEM_ID'].values[0]).ljust(8, ' '), 
          df2['GENRE'].values[0].ljust(20, ' '), 
          df2['ALLOWED_COUNTRIES'].values[0].ljust(20, ' '))

#### Record the Personalize endpoint ARN in Systems Manager Parameter Store

This stored value will be read in by the pre-created fronting setup and respond to user requests from their devices.

In [ ]:
try:
    describe_stack_resource_response = cf_client.describe_stack_resource(
        StackName=STACK_NAME,
        LogicalResourceId='SSMParameterStoreCampaignEndpointArn'
    )
    campaign_endpoint_paramter_store_name = describe_stack_resource_response["StackResourceDetail"]["PhysicalResourceId"]
    print('Paramter Store Name is:', campaign_endpoint_paramter_store_name)
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [ ]:
put_parameter_response = ssm_client.put_parameter(
    Name=campaign_endpoint_paramter_store_name,
    Value=campaign_arn,
    Type='String',
    Overwrite=True
)

print(put_parameter_response)

In [ ]:
get_parameter_response = ssm_client.get_parameter(
    Name=campaign_endpoint_paramter_store_name
)

if bool(get_parameter_response):
    print(get_parameter_response['Parameter']['Value'])

In [ ]:
try:
    describe_stack_filter_resource_response = cf_client.describe_stack_resource(
        StackName=STACK_NAME,
        LogicalResourceId='SSMParameterStoreFilterArn'
    )
    filter_arn_paramter_store_name = describe_stack_filter_resource_response["StackResourceDetail"]["PhysicalResourceId"]
    print('Filter Arn Name is:', filter_arn_paramter_store_name)
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [ ]:
put_filter_parameter_response = ssm_client.put_parameter(
    Name=filter_arn_paramter_store_name,
    Value=filter_arn,
    Type='String',
    Overwrite=True
)

print(put_filter_parameter_response)

In [ ]:
get_filter_parameter_response = ssm_client.get_parameter(
    Name=filter_arn_paramter_store_name
)

if bool(get_filter_parameter_response):
    print(get_filter_parameter_response['Parameter']['Value'])

## Review

Using the codes above you have successfully trained a deep learning model to generate item recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `2.View_Campaign_And_Interactions.ipynb`



## Notes for the Next Notebook:

There are a few values you will need for the next notebook, execute the cells below to store them so they can be copied and pasted into the next part of the exercise.

In [ ]:
%store campaign_arn

In [ ]:
%store dataset_group_arn

In [ ]:
%store solution_version_arn

In [ ]:
%store solution_arn

In [ ]:
%store interactions_dataset_arn

In [ ]:
%store items_data

In [ ]:
%store interactions_data

In [ ]:
%store items_dataset_arn

In [ ]:
%store interactions_schema_arn

In [ ]:
%store items_schema_arn

In [ ]:
%store bucket

In [ ]:
%store interactions_filename

In [ ]:
%store items_filename

In [ ]:
%store role_name

In [ ]:
%store user_id

In [ ]:
%store device

In [ ]:
%store cold_user_id

In [ ]:
%store cold_user_device

In [ ]:
%store filter_arn